# NLP Project - Image Captioning with Multilingual Text to Speech 
### Kirankumar Kanaje
### Harini Mohankumar
### Haravindan Jain
### Sameer Balkawade
### Kshitija Landge

# Import necessary libraries


In [43]:
# Import necessary libraries
import os
import pickle
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import warnings
from math import ceil
from collections import defaultdict
from tqdm.notebook import tqdm
import tensorflow as tf
from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Model
from tensorflow.keras.utils import to_categorical, plot_model
from tensorflow.keras.layers import Input, Dense, LSTM, Embedding, Dropout, concatenate, Bidirectional, Dot, Activation, RepeatVector, Multiply, Lambda
from nltk.translate.bleu_score import corpus_bleu

# Suppress warnings for cleaner output
warnings.filterwarnings('ignore')

"""
This script sets up a deep learning framework for an image captioning model,
leveraging pre-trained components and custom layers to process and generate
captions for images. It incorporates image processing, model building,
and evaluation stages.
"""

# Base libraries
# ------------------------------------------------------------------------------
# numpy for numerical operations
# matplotlib.pyplot for plotting graphs
# PIL.Image for image processing
# tqdm.notebook for progress bars in Jupyter Notebooks
# tensorflow as the deep learning framework
# nltk.translate for evaluation metrics (BLEU score)

'\nThis script sets up a deep learning framework for an image captioning model,\nleveraging pre-trained components and custom layers to process and generate\ncaptions for images. It incorporates image processing, model building,\nand evaluation stages.\n'

# Setting up Directories

In [40]:
"""
Set up the directory paths for the input and output of the project.
This set of code determines the current working directory, sets up relative paths to 
the dataset and the working directory where outputs will be stored.
It ensures that the script uses dynamic paths that work across different environments,
making the code more portable and easier to run on different machines without the need for manual path adjustments.
"""


# Get the current working directory
base_dir = os.getcwd()

# Define the relative path for the input directory where the dataset is located
INPUT_DIR = os.path.join(base_dir, 'Dataset', 'flickr8k')

# Define the relative path for the output directory where results and models will be saved
OUTPUT_DIR = os.path.join(base_dir, 'working')

# Load the pre-trained model

In [41]:
"""
Load and modify a pre-trained VGG16 model to use as a feature extractor.

This set of code loads the VGG16 model pre-trained on the ImageNet dataset and modifies it 
by removing the final classification layer. This modified model outputs the features from
the penultimate layer, which can be used for tasks that require feature extraction instead
of classification.

Returns:
Model: The modified VGG16 model with the last layer removed to serve as a feature extractor.
"""

# Load the original VGG16 model pre-trained on ImageNet
model = VGG16()

# Remove the last classification layer to convert the model into a feature extractor
# The output is now from the second to last layer
model = Model(inputs=model.inputs, outputs=model.layers[-2].output)

 # Print the summary of the modified model to verify the structure
print(model.summary())

Model: "model_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_7 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0   

# Feature Extraction

In [42]:
"""
Extracts features from images in a specified directory using a pre-trained model.
    
This function iterates through each image in the specified directory, preprocesses each image 
to fit the input requirements of the given pre-trained model, and uses the model to extract features.
These features are stored in a dictionary where keys are image IDs (without file extension).

Parameters:
directory (str): The path to the directory containing images.
model (Model): A pre-trained TensorFlow/Keras model used for extracting features.

Returns:
dict: A dictionary containing image IDs as keys and their corresponding extracted features as values.
"""


# Initialize an empty dictionary to store image features
image_features = {}

# Define the directory path where images are located
img_dir = os.path.join(INPUT_DIR, 'Images')

# Loop through each image in the directory
for img_name in tqdm(os.listdir(img_dir)):
    # Construct the full image path
    img_path = os.path.join(img_dir, img_name)
    # Load the image, resizing it to the required input size of the model (e.g., 224x224 for VGG16)
    image = load_img(img_path, target_size=(224, 224))
    # Convert image pixels to a numpy array
    image = img_to_array(image)
    # Reshape the array for batch processing compatibility with the model
    image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))
    # Preprocess the image data to match the input format expected by the model
    image = preprocess_input(image)
    # Use the model to extract features
    image_feature = model.predict(image, verbose=0)
    # Extract the image ID by removing the file extension
    image_id = img_name.split('.')[0]
    # Store the extracted features in the dictionary using the image ID as the key
    image_features[image_id] = image_feature

  0%|          | 0/8091 [00:00<?, ?it/s]

# Storing the Image Features Using Pickle


In [69]:
# Save the extracted image features to a file using pickle for later use
pickle.dump(image_features, open(os.path.join(OUTPUT_DIR, 'img_features.pkl'), 'wb'))

# Loading the Image Features Using Pickle

In [70]:
"""
Loads image features from a pickle file.

This function opens a specified pickle file in binary read mode and loads the
serialized image features stored in it. This is useful for retrieving pre-processed
data that was previously saved, avoiding the need to re-process data.

Parameters:
file_path (str): The path to the pickle file containing the serialized image features.

Returns: A dictionary containing the loaded image features, where keys are image IDs
and values are the extracted features.
"""


# Specify the file path for the pickled image features
pickle_file_path = os.path.join(OUTPUT_DIR, 'img_features.pkl')
 
# Open the file for reading in binary mode
with open(pickle_file_path, 'rb') as file:
    # Load the stored image features from the specified file using pickle
    loaded_features = pickle.load(file)

# Loading Caption Data

In [71]:
# Open the captions text file in read mode
with open(os.path.join(INPUT_DIR, 'captions.txt'), 'r') as file:
    # Skip the first line of the file, which might contain headers or unwanted text
    next(file)
    # Read the entire contents of the file after the first line into a single string
    captions_doc = file.read()

# Image to Caption Mapping

In [72]:
"""
Creates a mapping from image IDs to lists of captions.
    
This function processes a large string containing lines of text where each line consists of 
an image ID followed by captions. The function parses these lines to create a dictionary 
where each key is an image ID (without file extension) and the value is a list of associated captions.

Parameters:
 string containing all captions, each on a new line, typically read from a file.

Returns:
A dictionary mapping image IDs to lists of captions.
"""


# Initialize a defaultdict to hold a list of captions for each image ID
image_to_captions_mapping = defaultdict(list)

# Process each line in the document, splitting the document into lines first
for line in tqdm(captions_doc.split('\n')):
    # Split each line by comma to separate the image ID from the captions
    tokens = line.split(',')
    # Continue to the next line if there are not enough tokens
    if len(tokens) < 2:
        continue
    # The first token is the image ID, and the rest are part of the captions
    image_id, *captions = tokens
    # Remove the file extension from the image ID
    image_id = image_id.split('.')[0]
    # Join the captions parts into a single string
    caption = " ".join(captions)
    # Append the processed caption to the list of captions for this image ID
    image_to_captions_mapping[image_id].append(caption)

# Print the total number of captions
total_captions = sum(len(captions) for captions in image_to_captions_mapping.values())
print("Total number of captions:", total_captions)

  0%|          | 0/40456 [00:00<?, ?it/s]

Total number of captions: 40455


# Preprocessing Captions


In [73]:
"""
Processes the captions to standardize and clean the text.

This function iterates over each caption in the provided mapping dictionary,
performing several preprocessing steps to standardize the text. These steps include
converting to lowercase, removing non-alphabetical characters, reducing extra spaces,
and appending start and end tokens. This standardized format prepares captions for
further processing in natural language tasks.

Parameters:
A dictionary where keys are image IDs and values are lists of captions.

Returns:
None: The function modifies the captions in place.
"""

# Function for processing the captions
def clean(mapping):
    for key, captions in mapping.items():
        for i in range(len(captions)):
            # Take one caption at a time
            caption = captions[i]
            # Preprocessing steps
            # Convert to lowercase
            caption = caption.lower()
            # Remove any characters that are not alphabetic or spaces
            caption = ''.join(char for char in caption if char.isalpha() or char.isspace())
            # Remove extra spaces
            caption = caption.replace('\s+', ' ')
            # Add unique start and end tokens to the caption
            caption = 'startseq ' + ' '.join([word for word in caption.split() if len(word) > 1]) + ' endseq'
            # Update the caption in the list
            captions[i] = caption

In [57]:
# before preprocess of text
image_to_captions_mapping['1026685415_0431cbf574']
# preprocess the text
clean(image_to_captions_mapping)
# after preprocess of text
image_to_captions_mapping['1026685415_0431cbf574']


['startseq startseq startseq black dog carries green toy in his mouth as he walks through the grass endseq endseq endseq',
 'startseq startseq startseq black dog carrying something through the grass endseq endseq endseq',
 'startseq startseq startseq black dog has blue toy in its mouth endseq endseq endseq',
 'startseq startseq startseq dog in grass with blue item in his mouth endseq endseq endseq',
 'startseq startseq startseq wet black dog is carrying green toy through the grass endseq endseq endseq']

In [58]:
# Creating a List of All Captions
all_captions = [caption for captions in image_to_captions_mapping.values() for caption in captions]
all_captions[:10]

['startseq startseq startseq child in pink dress is climbing up set of stairs in an entry way endseq endseq endseq',
 'startseq startseq startseq girl going into wooden building endseq endseq endseq',
 'startseq startseq startseq little girl climbing into wooden playhouse endseq endseq endseq',
 'startseq startseq startseq little girl climbing the stairs to her playhouse endseq endseq endseq',
 'startseq startseq startseq little girl in pink dress going into wooden cabin endseq endseq endseq',
 'startseq startseq startseq black dog and spotted dog are fighting endseq endseq endseq',
 'startseq startseq startseq black dog and tricolored dog playing with each other on the road endseq endseq endseq',
 'startseq startseq startseq black dog and white dog with brown spots are staring at each other in the street endseq endseq endseq',
 'startseq startseq startseq two dogs of different breeds looking at each other on the road endseq endseq endseq',
 'startseq startseq startseq two dogs on pave

# Tokenizer

In [59]:
"""
Tokenizes the text, saves the tokenizer, reloads it, and calculates key metrics.
    
This function initializes a tokenizer, fits it on the provided captions to create
a vocabulary, and then saves the tokenizer to a file for later use. It reloads the 
tokenizer to ensure it works as expected and computes the maximum length of the captions
and the size of the vocabulary. These metrics are essential for setting up models 
in natural language processing tasks.

Parameters:
A list of all captions to be tokenized.

Returns:
None: The function prints the vocabulary size and maximum caption length.
"""

# Tokenizing the Text
tokenizer = Tokenizer()
tokenizer.fit_on_texts(all_captions)

# Save the tokenizer
with open('tokenizer.pkl', 'wb') as tokenizer_file:
    pickle.dump(tokenizer, tokenizer_file)

# Load the tokenizer
with open('tokenizer.pkl', 'rb') as tokenizer_file:
    tokenizer = pickle.load(tokenizer_file)

# Calculate maximum caption length
max_caption_length = max(len(tokenizer.texts_to_sequences([caption])[0]) for caption in all_captions)
vocab_size = len(tokenizer.word_index) + 1

# Print the results
print("Vocabulary Size:", vocab_size)
print("Maximum Caption Length:", max_caption_length)

Vocabulary Size: 8768
Maximum Caption Length: 38


# Train Test Split

In [60]:
# Creating a List of Image IDs
image_ids = list(image_to_captions_mapping.keys())
# Splitting into Training and Test Sets
split = int(len(image_ids) * 0.90)
train = image_ids[:split]
test = image_ids[split:]

In [61]:

"""
    Yields batches of data for training a model on image captioning tasks.

    This generator function creates batches of training data from image features and captions.
    Each batch contains image features, partial captions as input, and the next word in the caption as output.
    The function continues indefinitely, making it suitable for use with model training functions that require
    a generator for input.

    Parameters:
    data_keys (List[str]): List of image IDs that are used to retrieve data.
    image_to_captions_mapping (Dict[str, List[str]]): Dictionary mapping image IDs to lists of captions.
    features (Dict[str, np.ndarray]): Dictionary mapping image IDs to their corresponding feature vectors.
    tokenizer (Tokenizer): Tokenizer for converting text captions into sequences of integers.
    max_caption_length (int): Maximum length of caption sequences, used for padding.
    vocab_size (int): The size of the vocabulary, used for one-hot encoding.
    batch_size (int): Number of data samples per batch.

    Returns:
    Generator: Yields a tuple containing two lists (input features and input sequences) and one output list (target word).
"""

# Data generator function
def data_generator(data_keys, image_to_captions_mapping, features, tokenizer, max_caption_length, vocab_size, batch_size):
    # Lists to store batch data
    X1_batch, X2_batch, y_batch = [], [], []
    # Counter for the current batch size
    batch_count = 0

    while True:
        # Loop through each image in the current batch
        for image_id in data_keys: 
            # Get the captions associated with the current image
            captions = image_to_captions_mapping[image_id]

            # Loop through each caption for the current image
            for caption in captions:
                # Convert the caption to a sequence of token IDs
                caption_seq = tokenizer.texts_to_sequences([caption])[0]

                # Loop through the tokens in the caption sequence
                for i in range(1, len(caption_seq)):
                    # Split the sequence into input and output pairs
                    in_seq, out_seq = caption_seq[:i], caption_seq[i]

                    # Pad the input sequence to the specified maximum caption length
                    in_seq = pad_sequences([in_seq], maxlen=max_caption_length)[0]

                    # Convert the output sequence to one-hot encoded format
                    out_seq = to_categorical([out_seq], num_classes=vocab_size)[0]

                    # Append data to batch lists
                    X1_batch.append(features[image_id][0])  # Image features
                    X2_batch.append(in_seq)  # Input sequence
                    y_batch.append(out_seq)  # Output sequence

                    # Increase the batch counter
                    batch_count += 1

                    # If the batch is complete, yield the batch and reset lists and counter
                    if batch_count == batch_size:
                        X1_batch, X2_batch, y_batch = np.array(X1_batch), np.array(X2_batch), np.array(y_batch)
                        yield [X1_batch, X2_batch], y_batch
                        X1_batch, X2_batch, y_batch = [], [], []
                        batch_count = 0

# LSTM Model Training

Now that we have successfully prepared our image features, we are poised to embark on an exciting phase: training our LSTM model to generate captions. This process is the heart of our project, where we combine the power of neural networks with natural language processing techniques. By training the LSTM model, we aim to teach it to understand the context of images and creatively produce accurate and descriptive captions. This step is not just about feeding data into a model; it's about fine-tuning the parameters and optimizing the network to capture the nuances of language associated with visual elements, thereby bringing the power of image understanding and linguistic generation together.


In [62]:
"""
    Constructs a neural network model with attention mechanism for image captioning.
    
    This function creates a model that incorporates both image features and textual input
    using an attention mechanism to focus on relevant parts of the image while generating
    captions. The model architecture includes:
    - An encoder for image features with dropout and dense layers.
    - A sequence model for processing captions with embedding and LSTM layers.
    - An attention mechanism that combines features from both encoders to enhance the
      caption generation process.

    Parameters:
    input_dim (int): Dimension of the input image feature vector.
    vocab_size (int): The size of the vocabulary for the captions.
    max_caption_length (int): The maximum length of the captions.

    Returns:
    Model: A compiled Keras model ready for training.
"""

# Encoder model
inputs1 = Input(shape=(4096,))
fe1 = Dropout(0.5)(inputs1)
fe2 = Dense(256, activation='relu')(fe1)
fe2_projected = RepeatVector(max_caption_length)(fe2)
fe2_projected = Bidirectional(LSTM(256, return_sequences=True))(fe2_projected)

# Sequence feature layers
inputs2 = Input(shape=(max_caption_length,))
se1 = Embedding(vocab_size, 256, mask_zero=True)(inputs2)
se2 = Dropout(0.5)(se1)
se3 = Bidirectional(LSTM(256, return_sequences=True))(se2)

# Apply attention mechanism using Dot product
attention = Dot(axes=[2, 2])([fe2_projected, se3])  # Calculate attention scores

# Softmax attention scores
attention_scores = Activation('softmax')(attention)

# Apply attention scores to sequence embeddings
attention_context = Lambda(lambda x: tf.einsum('ijk,ijl->ikl', x[0], x[1]))([attention_scores, se3])

# Sum the attended sequence embeddings along the time axis
context_vector = tf.reduce_sum(attention_context, axis=1)

# Decoder model
decoder_input = concatenate([context_vector, fe2], axis=-1)
decoder1 = Dense(256, activation='relu')(decoder_input)
outputs = Dense(vocab_size, activation='softmax')(decoder1)

# Create the model
model = Model(inputs=[inputs1, inputs2], outputs=outputs)
model.compile(loss='categorical_crossentropy', optimizer='adam')


2024-05-03 14:42:12.861308: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2024-05-03 14:42:12.862225: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2024-05-03 14:42:12.862767: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

In [63]:
"""
    Trains an image captioning model using specified training and validation datasets.

    This function sets up training and validation generators, computes the necessary steps per epoch
    for both training and validation phases, and executes the training process over a specified number of epochs.
    
    Parameters:
    model (Model): The image captioning model to be trained.
    train_keys (List[str]): List of image IDs used for training.
    test_keys (List[str]): List of image IDs used for validation.
    image_to_captions_mapping (Dict[str, List[str]]): Dictionary mapping image IDs to lists of captions.
    features (Dict[str, np.ndarray]): Dictionary mapping image IDs to their corresponding feature vectors.
    tokenizer (Tokenizer): Tokenizer for converting text captions into sequences of integers.
    max_caption_length (int): Maximum length of caption sequences, used for padding.
    vocab_size (int): The size of the vocabulary for the captions.
    batch_size (int): Number of data samples per batch.
    epochs (int): Number of epochs to train the model.

    Returns:
    None: The model is trained in-place and can be used or saved post training.
 """


# Set the number of epochs, batch size
epochs = 60
batch_size = 16

# Calculate the steps_per_epoch based on the number of batches in one epoch
steps_per_epoch = ceil(len(train) / batch_size)
validation_steps = ceil(len(test) / batch_size)  # Calculate the steps for validation data

# Loop through the epochs for training
for epoch in range(epochs):
    print(f"Epoch {epoch+1}/{epochs}")
    
    # Set up data generators
    train_generator = data_generator(train, image_to_captions_mapping, loaded_features, tokenizer, max_caption_length, vocab_size, batch_size)
    test_generator = data_generator(test, image_to_captions_mapping, loaded_features, tokenizer, max_caption_length, vocab_size, batch_size)
    
    model.fit(train_generator, epochs=1, steps_per_epoch=steps_per_epoch,
          validation_data=test_generator, validation_steps=validation_steps,
          verbose=1)

Epoch 1/60


2024-05-03 14:43:25.508713: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]
2024-05-03 14:43:27.602503: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2024-05-03 14:43:27.603782: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_gra

456/456 [==============================] - ETA: 0s - loss: 4.6607

2024-05-03 14:44:13.064538: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]
2024-05-03 14:44:14.015892: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2024-05-03 14:44:14.016960: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_gra

456/456 [==============================] - 51s 99ms/step - loss: 4.6607 - val_loss: 4.4728
Epoch 2/60
  2/456 [..............................] - ETA: 41s - loss: 3.9976

...

456/456 [==============================] - 43s 94ms/step - loss: 0.6027 - val_loss: 10.9385
Epoch 60/60
  2/456 [..............................] - ETA: 41s - loss: 0.6294

2024-05-03 15:26:21.588418: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


456/456 [==============================] - ETA: 0s - loss: 0.5753

2024-05-03 15:27:02.521694: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


456/456 [==============================] - 43s 93ms/step - loss: 0.5753 - val_loss: 11.4690


# Saving the Model

In [65]:
# Save the model
model.save(OUTPUT_DIR+'/mymodel.h5')

# Captions Generation

In [66]:
def get_word_from_index(index, tokenizer):
    return next((word for word, idx in tokenizer.word_index.items() if idx == index), None)

In [74]:
"""
    Generates a caption for an image using a trained model and image features.

    This function initializes a caption with a start sequence token, then iteratively predicts
    the next word in the caption until the end sequence token is predicted or the maximum
    caption length is reached. Each word is predicted based on the current state of the caption,
    using the model to generate a probability distribution over possible next words.

    Parameters:
    model (Model): The trained image captioning model.
    image_features (np.ndarray): The extracted features of the image to be captioned.
    tokenizer (Tokenizer): The tokenizer used for encoding and decoding words to indices.
    max_caption_length (int): The maximum length of the caption to be generated.

    Returns:
    str: The generated caption for the image.
"""

def predict_caption(model, image_features, tokenizer, max_caption_length):
    # Initialize the caption sequence
    caption = 'startseq'
    
    # Generate the caption
    for _ in range(max_caption_length):
        # Convert the current caption to a sequence of token indices
        sequence = tokenizer.texts_to_sequences([caption])[0]
        # Pad the sequence to match the maximum caption length
        sequence = pad_sequences([sequence], maxlen=max_caption_length)
        # Predict the next word's probability distribution
        yhat = model.predict([image_features, sequence], verbose=0)
        # Get the index with the highest probability
        predicted_index = np.argmax(yhat)
        # Convert the index to a word
        predicted_word = get_word_from_index(predicted_index, tokenizer)
        
        # Append the predicted word to the caption
        caption += " " + predicted_word
        
        # Stop if the word is None or if the end sequence tag is encountered
        if predicted_word is None or predicted_word == 'endseq':
            break
    
    return caption

# Generating BLEU Scores

In [68]:
"""
    Evaluates the image captioning model using the BLEU score metric.

    This function iterates over a set of test image keys, generates captions using the trained model,
    and compares these generated captions against the actual captions. It calculates the BLEU scores
    for assessing the quality of the generated captions.

    Parameters:
    model (Model): The trained image captioning model.
    test_keys (List[str]): List of image IDs used for testing.
    image_to_captions_mapping (Dict[str, List[str]]): Dictionary mapping image IDs to lists of actual captions.
    features (Dict[str, np.ndarray]): Dictionary mapping image IDs to their corresponding feature vectors.
    tokenizer (Tokenizer): The tokenizer used for encoding and decoding words to indices.
    max_caption_length (int): The maximum length of the captions.

    Returns:
    None: Outputs the BLEU scores directly to standard output.
"""


# Initialize lists to store actual and predicted captions
actual_captions_list = []
predicted_captions_list = []

# Loop through the test data
for key in tqdm(test):
    # Get actual captions for the current image
    actual_captions = image_to_captions_mapping[key]
    # Predict the caption for the image using the model
    predicted_caption = predict_caption(model, loaded_features[key], tokenizer, max_caption_length)
    
    # Split actual captions into words
    actual_captions_words = [caption.split() for caption in actual_captions]
    # Split predicted caption into words
    predicted_caption_words = predicted_caption.split()
    
    # Append to the lists
    actual_captions_list.append(actual_captions_words)
    predicted_captions_list.append(predicted_caption_words)

# Calculate BLEU score
print("BLEU-1: %f" % corpus_bleu(actual_captions_list, predicted_captions_list, weights=(1.0, 0, 0, 0)))
print("BLEU-2: %f" % corpus_bleu(actual_captions_list, predicted_captions_list, weights=(0.5, 0.5, 0, 0)))

  0%|          | 0/810 [00:00<?, ?it/s]

2024-05-03 15:27:52.006612: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2024-05-03 15:27:52.007891: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2024-05-03 15:27:52.008895: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

BLEU-1: 0.417114
BLEU-2: 0.277707
